### Demonstration Overview: Translating an Allele Profile into a  VRS (version 2.0) object

This notebook demonstrates the conversion of an **HL7 FHIR AlleleProfile** into a **GA4GH VRS (Version 2.0)** representation. Using example data, an `AlleleProfile` object is generated and subsequently translated into a VRS allele via the `translate_allele_to_vrs` method from the `VrsFhirAlleleTranslator` class. This method facilitates a one-way transformation from **FHIR AlleleProfiles** to **VRS Alleles**.

For a more comprehensive understanding of the **HL7 FHIR Allele**, we recommend reviewing the [FHIR Allele Demo Notebook](../schema/allele_profile_demo.ipynb) notebook as well as the [HL7 FHIR MolecularDefinition Documentation](https://build.fhir.org/branches/cg-im-moldef_work_in_progress_2/moleculardefinition.html).


### Prerequisites and Setup

To support the one way transformation from FHIR Alelle to VRS Allele of this demonstration, we set up the environment by importing the necessary libraries and modules. These include:

1. **Custom Project Modules**:
   - `Allele` from `profiles.allele`: A data structure representing an HL7 FHIR Allele
   - `VrsFhirAlleleTranslator` from `translators.vrs_fhir_translator`: A translation component for converting VRS alleles into Allele Profile.

In [1]:
from profiles.allele import Allele as FhirAllele
from translators.vrs_fhir_translator import VrsFhirAlleleTranslator
from ga4gh.vrs.dataproxy import create_dataproxy


In [2]:
dp = create_dataproxy(uri="seqrepo+file:///usr/local/share/seqrepo/2024-12-20")
allele_translator = VrsFhirAlleleTranslator(dp=dp)

In [3]:
example_substitution = {
    "resourceType": "MolecularDefinition",
    "contained": [
        {
            # This is the id for the Sequence
            "id": "ref-to-nc000002",
            "resourceType": "MolecularDefinition",
            "moleculeType": {
                "coding": [
                    {
                        "system": "http://hl7.org/fhir/sequence-type",
                        "code": "dna",
                        "display": "DNA Sequence",
                    }
                ]
            },
            # Remember Sequence doesn't include a location or memberState.
            "representation": [
                {
                    "code": [
                        {
                            "coding": [
                                {
                                    "system": "http://www.ncbi.nlm.nih.gov/refseq",
                                    # This is where we need to capture the reference sequence
                                    "code": "NC_000002.12",
                                }
                            ]
                        }
                    ]
                }
            ],
        }
    ],
    "moleculeType": {
        "coding": [
            {
                "system": "http://hl7.org/fhir/sequence-type",
                "code": "dna",
                "display": "DNA Sequence",
            }
        ]
    },
    "location": [
        {
            "sequenceLocation": {
                "sequenceContext": {
                    # This indicates that we are referencing the sequence mentioned above within the contained section.
                    "reference": "#ref-to-nc000002",
                    "type": "MolecularDefinition",
                    "display": "NC_000002.12",
                },
                "coordinateInterval": {
                    "coordinateSystem": {
                        "system": {
                            "coding": [
                                {
                                    "system": "http://loinc.org",
                                    "code": "LA30100-4",
                                    # Must only contain the system's coordinate for translation
                                    "display": "0-based interval counting",
                                }
                            ],
                            "text": "0-based interval counting",
                        }
                    },
                    # Example needs to contain the startQuantity for translation
                    "startQuantity": {"value": 27453448},
                    # Example needs to contain the endQuantity for translation
                    "endQuantity": {"value": 27453449},
                },
            }
        }
    ],
    "representation": [
        {
            "focus": {
                "coding": [
                    {
                        # Example needs to contain system
                        "system": "http://hl7.org/fhir/moleculardefinition-focus",
                        # Example needs to contain code
                        "code": "allele-state",
                        "display": "Allele State",
                    }
                ]
            },
            "literal": {
                # Example needs to contain the literal value for translation
                "value": "T"
            },
        }
    ],
}

In [4]:
example_allele_substitution = FhirAllele(**example_substitution)
example_allele_substitution.model_dump()

{'resourceType': 'MolecularDefinition',
 'contained': [{'resourceType': 'MolecularDefinition',
   'id': 'ref-to-nc000002',
   'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
      'code': 'dna',
      'display': 'DNA Sequence'}]},
   'representation': [{'code': [{'coding': [{'system': 'http://www.ncbi.nlm.nih.gov/refseq',
         'code': 'NC_000002.12'}]}]}]}],
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': '#ref-to-nc000002',
     'type': 'MolecularDefinition',
     'display': 'NC_000002.12'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}],
       'text': '0-based interval counting'}},
     'startQuantity': {'value': 27453448.0},
     'endQuantity': {'value': 27453449.0}}}}],

### Normalized VRS allele translation

In [5]:
vrs_example_allele_substitution_norm = allele_translator.translate_allele_to_vrs(example_allele_substitution)
vrs_example_allele_substitution_norm.model_dump(exclude_none=True)

{'id': 'ga4gh:VA.xfKU4c8mG_yegL5ZOL26JDiznySNkoMl',
 'type': 'Allele',
 'digest': 'xfKU4c8mG_yegL5ZOL26JDiznySNkoMl',
 'location': {'id': 'ga4gh:SL.y0ckc1_lhMYKnh0f6FAEoEpgHyfX13OW',
  'type': 'SequenceLocation',
  'digest': 'y0ckc1_lhMYKnh0f6FAEoEpgHyfX13OW',
  'sequenceReference': {'type': 'SequenceReference',
   'refgetAccession': 'SQ.pnAqCRBrTsUoBghSD1yp_jXWSmlbdh4g'},
  'start': 27453448,
  'end': 27453449},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'T'}}

### Non-normalized VRS allele translation

In [6]:
vrs_example_allele_substitution_unnorm = allele_translator.translate_allele_to_vrs(example_allele_substitution,normalize=False)
vrs_example_allele_substitution_unnorm.model_dump(exclude_none=True)

{'type': 'Allele',
 'location': {'type': 'SequenceLocation',
  'sequenceReference': {'type': 'SequenceReference',
   'refgetAccession': 'SQ.pnAqCRBrTsUoBghSD1yp_jXWSmlbdh4g'},
  'start': 27453448,
  'end': 27453449},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'T'}}

### Example 2

In [7]:
example_insertion = {
    "resourceType": "MolecularDefinition",
    "contained": [
        {
            "resourceType": "MolecularDefinition",
            # This is the id for the Sequence
            "id": "ref-to-nc000001",
            "moleculeType": {
                "coding": [
                    {
                        "system": "http://hl7.org/fhir/sequence-type",
                        "code": "dna",
                        "display": "DNA Sequence",
                    }
                ]
            },
            # Remember Sequence doesn't include a location or memberState.
            "representation": [
                {
                    "code": [
                        {
                            "coding": [
                                {
                                    "system": "http://www.ncbi.nlm.nih.gov/refseq",
                                    # This is where we need to capture the reference sequence
                                    "code": "NC_000001.11",
                                }
                            ]
                        }
                    ]
                }
            ],
        }
    ],
    "moleculeType": {
        "coding": [
            {
                "system": "http://hl7.org/fhir/sequence-type",
                "code": "dna",
                "display": "DNA Sequence",
            }
        ]
    },
    "location": [
        {
            "sequenceLocation": {
                "sequenceContext": {
                    # This indicates that we are referencing the sequence mentioned above within the contained section.
                    "reference": "#ref-to-nc000001",
                    "type": "MolecularDefinition",
                },
                "coordinateInterval": {
                    "coordinateSystem": {
                        "system": {
                            "coding": [
                                {
                                    "system": "http://loinc.org",
                                    "code": "LA30100-4",
                                    # Must only contain the system's coordinate for translation
                                    "display": "0-based interval counting",
                                }
                            ],
                            "text": "0-based interval counting",
                        }
                    },
                    # Example needs to contain the startQuantity for translation
                    "startQuantity": {"value": 113901365},
                    # Example needs to contain the endQuantity for translation
                    "endQuantity": {"value": 113901365},
                },
            }
        }
    ],
    "representation": [
        {
            "focus": {
                "coding": [
                    {
                        # Example needs to contain system
                        "system": "http://hl7.org/fhir/moleculardefinition-focus",
                        # Example needs to contain code
                        "code": "allele-state",
                        "display": "Allele State",
                    }
                ]
            },
            "literal": {
                # Example needs to contain the literal value for translation
                "value": "ATA"
            },
        }
    ],
}


In [8]:
example_allele_insertion = FhirAllele(**example_insertion)
example_allele_insertion.model_dump()

{'resourceType': 'MolecularDefinition',
 'contained': [{'resourceType': 'MolecularDefinition',
   'id': 'ref-to-nc000001',
   'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
      'code': 'dna',
      'display': 'DNA Sequence'}]},
   'representation': [{'code': [{'coding': [{'system': 'http://www.ncbi.nlm.nih.gov/refseq',
         'code': 'NC_000001.11'}]}]}]}],
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': '#ref-to-nc000001',
     'type': 'MolecularDefinition'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}],
       'text': '0-based interval counting'}},
     'startQuantity': {'value': 113901365.0},
     'endQuantity': {'value': 113901365.0}}}}],
 'representation': [{'focus':

In [9]:
vrs_example_allele_insertion = allele_translator.translate_allele_to_vrs(example_allele_insertion)
vrs_example_allele_insertion.model_dump(exclude_none=True)

{'id': 'ga4gh:VA.3edM6TTGAmx8DnPV-uzA6IYlAfatAP2s',
 'type': 'Allele',
 'digest': '3edM6TTGAmx8DnPV-uzA6IYlAfatAP2s',
 'location': {'id': 'ga4gh:SL.OUMCiUkn_AGlFuFCFTdfppig932_HV2k',
  'type': 'SequenceLocation',
  'digest': 'OUMCiUkn_AGlFuFCFTdfppig932_HV2k',
  'sequenceReference': {'type': 'SequenceReference',
   'refgetAccession': 'SQ.Ya6Rs7DHhDeg7YaOSg1EoNi3U_nQ9SvO'},
  'start': 113901365,
  'end': 113901365},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'ATA'}}

### Conclusion

In this notebook, we demonstrated how to translate HL7 FHIR Allele into GA4GH VRS alleles. We began by using Python’s `**` unpacking syntax to generate a class instance directly from the complete JSON structure, and then converted the FHIR Allele object into a VRS Allele object using the `VrsFhirAlleleTranslator` class.

If you would like to learn more about building FHIR Allele step-by-step, we recommend reviewing the [FHIR Allele Demo Notebook](../schema/allele_profile_demo.ipynb) notebook.

We recognize that the HL7 FHIR MolecularDefinition schema is continuously evolving, which may affect the structure of FHIR Allele. As the schema changes, this code will also continue to evolve to maintain compatibility.
